In [4]:
import numpy as np 
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords
from gensim.models import Word2Vec 
from scipy import spatial
import networkx as nx
import csv

In [5]:
df=pd.read_csv("/kaggle/input/medium-articles/medium_articles.csv")
df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [6]:
df=df[['title','text']]
df.shape

(192368, 2)

In [7]:
df=df.drop_duplicates()
df.shape

(190827, 2)

In [9]:
df['text']=df['text'].str.replace('\n','<|n|>')

In [10]:
sample_blog=df['text'][10]
print(sample_blog)

Photo credit: Leo Leung<|n|><|n|>People want to know why you do what you do.<|n|><|n|>If you’re an entrepreneur, your investors, customers, and employees want to know why you founded your company. Apply for a job, and recruiters ask about your career path. If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories? Because they reveal character. Superman is a good guy because his planet exploded, his parents sent him to Earth, and the Kents raised him to fight evil. Steve Jobs became a trusted CEO, in part, by bouncing back from failure. My dad thrived as a home builder, but only after terminating a partnership with a prominent colleague whose practices he found unethical.<|n|><|n|>In other words:<|n|><|n|>Your origin story helps people decide whether or not to trust you.<|n|><|n|>How to Craft a Powerful Origin Story: The 5 Essential Elements<|n|><|n|>In strategic messaging and positioning engagements with CEOs, as well as business st

In [13]:
sentences=sent_tokenize(sample_blog)
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')

sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)

sentence_embeddings=[[w2v.wv.get_vector(word)[0] for word in words ] for words in sentence_tokens]
max_len=max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]



In [15]:
similarity_matrix = np.zeros([len(sentence_tokens),len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
        
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph,max_iter=600)

top_sentence ={sentence:scores[index] for index,sentence in enumerate(sentences)}
sentNeeded = round(0.25 * len(sentences))-1
top=dict(sorted(top_sentence.items(),key=lambda x:x[1],reverse =True)[:sentNeeded])

summary =""
for sent in sentences:
    if sent in top.keys():
        summary += sent
print(summary)

Apply for a job, and recruiters ask about your career path.If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories?Steve Jobs became a trusted CEO, in part, by bouncing back from failure.Very frequently, another person illuminates a new way forward.<|n|><|n|>Can you pinpoint the event that set you on your current path?The sign says, “For anyone who wants to tell a compelling story,” and it’s pointing to a book about screenwriting.Basically, describe what happened next:<|n|><|n|>That night, I read the book from cover to cover.It’s written in the language of feature films (three act structure, etc.I rewrite my pitch and send it to a new batch of investors.<|n|><|n|>4.“Until finally…”<|n|><|n|>What was the turning point — the moment when you really committed to the new path?Pay particular attention to where they lean in.<|n|><|n|>Also, ask others to tell you their origin stories.


In [16]:
count=0
def generateSummary(blog):
    global count
    count+=1
    print("Summarising blog",count)
    try:
        sentences=sent_tokenize(blog)
        sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
        stop_words = stopwords.words('english')

        sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
        w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)

        sentence_embeddings=[[w2v.wv.get_vector(word)[0] for word in words ] for words in sentence_tokens]
        max_len=max([len(tokens) for tokens in sentence_tokens])
        sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
        similarity_matrix = np.zeros([len(sentence_tokens),len(sentence_tokens)])
        for i,row_embedding in enumerate(sentence_embeddings):
            for j,column_embedding in enumerate(sentence_embeddings):
                similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)

        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph,max_iter=600)

        top_sentence ={sentence:scores[index] for index,sentence in enumerate(sentences)}
        sentNeeded = round(0.25 * len(sentences))-1
        top=dict(sorted(top_sentence.items(),key=lambda x:x[1],reverse =True)[:sentNeeded])

        summary =""
        for sent in sentences:
            if sent in top.keys():
                summary += sent
        return summary
    except:
        return float("NaN")



In [ ]:
import math
filename = "articlesSet.csv"
fields=['title','summary','content']


with open(filename,'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    
    def callback(row):
        summary=generateSummary(row['text'])
        if(type(summary)!=str):
            return 
        rows =[row['title'],summary,row['text']]
        csvwriter.writerow(rows)
    df.apply(callback,axis=1)